In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

OUTPUT_DIR = "./output_task2"

def main():
    if not os.path.exists(OUTPUT_DIR):
        print("Папка с результатами не найдена.")
        return

    try:
        subdirs = [os.path.join(OUTPUT_DIR, d) for d in os.listdir(OUTPUT_DIR) if d.startswith("report")]
        if not subdirs:
            print("Отчетов пока нет.")
            return
            
        latest_dir = max(subdirs, key=os.path.getmtime)
        print(f"Читаем последний отчет из: {latest_dir}")
    except Exception as e:
        print(f"Ошибка поиска файлов: {e}")
        return

    # Запускаем Spark только для чтения
    spark = SparkSession.builder \
        .appName("ResultViewer") \
        .master("local[*]") \
        .getOrCreate()
    
    spark.sparkContext.setLogLevel("ERROR")

    # Читаем CSV и сортируем
    df = spark.read.option("header", "true").csv(latest_dir)
    
    print("\n" + "="*30)
    print("ИТОГОВЫЙ ТОП-10 СЛОВ:")
    print("="*30)
    
    df.orderBy(col("count").cast("int").desc()).show(10, truncate=False)

    spark.stop()

if __name__ == "__main__":
    main()

Читаем последний отчет из: ./output_task2/report_20260110_232300

ИТОГОВЫЙ ТОП-10 СЛОВ:
+------------+-----+
|cleaned_word|count|
+------------+-----+
|Пайор       |4    |
|Челс        |4    |
|Эв          |4    |
|Атлетик     |4    |
|Тв          |3    |
|Доз         |3    |
|Футбол      |3    |
|Мадрид      |3    |
|Вот         |2    |
|Эт          |2    |
+------------+-----+
only showing top 10 rows

